In [1]:
%gui qt

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from ipywidgets import interact
import seaborn as sns
sns.set_context('poster')

In [4]:
%load_ext autoreload

In [5]:
%autoreload 2

### FPGA and Probe Configuration

./spkDect_Sort_Demo.ipynb

In [6]:
from spiketag.base import ProbeFactory, bload
from spiketag.mvc  import Sorter
from spiketag.fpga import xike_config

#### Probe Parameters

In [7]:
nCh = 160
fs = 25000.
tetrodes = ProbeFactory.genTetrodeProbe(fs=fs, n_ch=nCh)
tetrodes.fromfile('./open-ephys-load/40 tetrode_channel_map')

In [8]:
tetrodes.reorder_by_chip = True
tetrodes._nchips = 5

In [9]:
tetrodes.ch_hash(7)

array([6, 7, 8, 9])

### Sorting

In [10]:
!cp spk.bin mua.bin.spk

In [75]:
sorter = Sorter('/disk0/testbench/mua.bin', 
                probe=tetrodes, 
                fet_method='weighted-pca', fetlen=6,
                clu_method='hdbscan',
                n_jobs=24)

2017-10-09 15:39:53,460 - spiketag - INFO - load mua data
2017-10-09 15:39:53,462 - spiketag - INFO - #############  load data  ###################
2017-10-09 15:39:53,463 - spiketag - INFO - /disk0/testbench/mua.bin loaded, it contains: 
2017-10-09 15:39:53,464 - spiketag - INFO - 231959 * 160 points (148453760 bytes) 
2017-10-09 15:39:53,465 - spiketag - INFO - 160 channels with sampling rate of 25000.0000 
2017-10-09 15:39:53,466 - spiketag - INFO - 9.278 secs (0.155 mins) of data
2017-10-09 15:39:53,468 - spiketag - INFO - #############################################
2017-10-09 15:39:53,577 - spiketag - INFO - reordered with nchips=5 and nch_perchip=32
2017-10-09 15:39:53,660 - spiketag - INFO - raw data have 10862 spks
2017-10-09 15:39:53,662 - spiketag - INFO - removing all spks on group which len(spks) less then fetlen
2017-10-09 15:39:53,670 - spiketag - INFO - removed all spks on these groups: {}
2017-10-09 15:39:53,672 - spiketag - INFO - grouping spike time
2017-10-09 15:39

In [76]:
sorter.run()

In [77]:
sorter.model.spk[7].shape

(465, 19, 4)

In [78]:
sorter.model.spk.W

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.], dtype=float32)

## spikes stack and fet

In [79]:
r = sorter.model.spk[7]

In [80]:
x = r.transpose(0,2,1).ravel().reshape(-1, r.shape[1]*r.shape[2])
x.shape

(465, 76)

In [81]:
fet = sorter.model.fet[7]
fet.shape

(465, 6)

### Test for individual spikes

In [82]:
from spiketag.base.SPK import _construct_transformer, _transformer

In [83]:
pca_comp, shift, scale = _construct_transformer(x)

In [84]:
_fet = _transformer(x, pca_comp, shift, scale)

In [85]:
np.allclose(_fet, fet)

True

In [86]:
print x.shape, pca_comp.shape, shift.shape, scale.shape

(465, 76) (76, 6) (6,) ()


76 dims -> 6 dims -> shift -> scale

In [87]:
_fet[280]

array([ 0.02168342, -0.09064207,  0.0199933 ,  0.02243792,  0.01181959,
       -0.01131057])

In [88]:
fet[280]

array([ 0.02168342, -0.09064207,  0.0199933 ,  0.02243792,  0.01181959,
       -0.01131057])